# Run Differential Model - Random Forest

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn import metrics

pd.set_option('display.max_columns', 2105)
pd.set_option('display.max_rows',500)

In [2]:
games = pd.read_csv('../Data/MLBgames.csv')
games.head()

,date,num_of_games,day,visitor_team,v_league,v_game_num,home_team,h_league,h_game_num,v_runs,h_runs,total_outs,day_night,park_id,v_abs,v_hits,v_2b,v_3b,v_hr,v_rbis,v_sacbunt,v_sacfly,v_hbp,v_bb,v_ibb,v_k,v_sb,v_cs,v_gidp,v_ci,v_lob,v_p_used,v_ind_er,v_team_er,v_wildp,v_balk,v_putouts,v_assts,v_errors,v_passed_ball,v_dp,v_trip,h_abs,h_hits,h_2b,h_3b,h_hr,h_rbis,h_sacbunt,h_sacfly,h_hbp,h_bb,h_ibb,h_k,h_sb,h_cs,h_gidp,h_ci,h_lob,h_p_used,h_ind_er,h_team_er,h_wildp,h_balk,h_putouts,h_assts,h_errors,h_passed_ball,h_dp,h_trip,v_mngr_id,v_mngr_name,h_mngr_id,h_mngr_name,win_pitcher_id,win_pitcher_name,lose_pitch_id,lose_pitch_name,sve_pitch_id,sve_pitch_name,v_starter_id,v_starter_name,h_starter_id,h_starter_name,v_batter1_id,v_batter1_name,v_batter1_pos,v_batter2_id,v_batter2_name,v_batter2_pos,v_batter3_id,v_batter3_name,v_batter3_pos,v_batter4_id,v_batter4_name,v_batter4_pos,v_batter5_id,v_batter5_name,v_batter5_pos,v_batter6_id,v_batter6_name,v_batter6_pos,v_batter7_id,v_batter7_name,v_batter7_pos,v_batter8_id,v_batter8_name,v_batter8_pos,v_batter9_id,v_batter9_name,v_batter9_pos,h_batter1_id,h_batter1_name,h_batter1_pos,h_batter2_id,h_batter2_name,h_batter2_pos,h_batter3_id,h_batter3_name,h_batter3_pos,h_batter4_id,h_batter4_name,h_batter4_pos,h_batter5_id,h_batter5_name,h_batter5_pos,h_batter6_id,h_batter6_name,h_batter6_pos,h_batter7_id,h_batter7_name,h_batter7_pos,h_batter8_id,h_batter8_name,h_batter8_pos,h_batter9_id,h_batter9_name,h_batter9_pos,run_diff,W,L,total,v_ba,h_ba,v_obp,h_obp,v_1b,h_1b,v_tbs,h_tbs,v_slg,h_slg,v_ops,h_ops
0,19890403,0,Mon,BOS,NaN,1,BAL,0,1,4,5,64,0,BAL11,40,10,3,0,1,4,2,0,0,3,1,5,1,1,0,0,8,5,5,5,0,0,31,14,0,0,2,0,40,12,2,0,1,5,0,0,0,4,0,7,1,1,2,0,8,3,4,4,0,0,33,13,0,1,0,0,morgj101,Joe Morgan,robif103,Frank Robinson,holtb001,Brian Holton,stanb001,Bob Stanley,None,(none),clemr001,Roger Clemens,schmd001,Dave J. Schmidt,boggw001,Wade Boggs,5,barrm001,Marty Barrett,4,evand002,Dwight Evans,9,greem001,Mike Greenwell,7,burke001,Ellis Burks,8,ricej001,Jim Rice,10,esasn001,Nick Esasky,3,gedmr001,Rich Gedman,2,reedj003,Jody Reed,6,andeb001,Brady Anderson,8,bradp001,Phil Bradley,7,finls001,Steve Finley,9,ripkc001,Cal Ripken,6,sheel001,Larry Sheets,10,tettm001,Mickey Tettleton,2,trabj001,Jim Traber,3,wortc001,Craig Worthington,5,gonzr001,Rene Gonzales,4,1,BAL,BOS,9,0.250,0.300,0.350,0.400,6,9,16,17,0.400,0.425,0.750,0.825
1,19890403,0,Mon,MIL,NaN,1,CLE,0,1,1,2,51,0,CLE07,30,5,0,0,0,1,1,0,0,0,0,7,0,1,0,0,3,1,1,1,0,0,24,10,1,0,1,0,31,8,1,0,0,2,0,0,0,1,0,4,0,0,0,0,6,2,1,1,0,0,27,9,0,0,0,0,trebt801,Tom Trebelhorn,edwad102,Doc Edwards,swing001,Greg Swindell,augud001,Don August,joned001,Doug Jones,augud001,Don August,swing001,Greg Swindell,polig001,Gus Polidor,5,shefg001,Gary Sheffield,6,younr001,Robin Yount,8,deerr001,Rob Deer,9,bragg001,Glenn Braggs,7,frant001,Terry Francona,3,meyej001,Joey Meyer,10,surhb001,B.J. Surhoff,2,gantj001,Jim Gantner,4,mcdoo001,Oddibe McDowell,7,browj001,Jerry Browne,4,cartj001,Joe Carter,8,obrip001,Pete O'Brien,3,snydc001,Cory Snyder,9,clard001,Dave Clark,10,jacob001,Brook Jacoby,5,allaa001,Andy Allanson,2,fermf001,Felix Fermin,6,1,CLE,MIL,3,0.167,0.258,0.167,0.290,5,7,5,9,0.167,0.290,0.334,0.580
2,19890403,0,Mon,TOR,NaN,1,KCA,0,1,4,3,54,0,KAN06,36,10,3,1,0,4,0,1,0,1,0,5,0,0,0,0,7,3,3,3,0,0,27,9,0,0,1,0,34,9,4,0,0,3,1,0,0,1,0,6,1,0,1,0,6,2,4,4,0,0,27,10,0,0,0,0,willj107,Jimy Williams,wathj001,John Wathan,key-j001,Jimmy Key,gubim001,Mark Gubicza,henkt001,Tom Henke,key-j001,Jimmy Key,gubim001,Mark Gubicza,mosel001,Lloyd Moseby,8,mullr001,Rance Mulliniks,5,fernt001,Tony Fernandez,6,bellg001,George Bell,7,mcgrf001,Fred McGriff,3,barfj001,Jesse Barfield,9,white002,Ernie Whitt,2,lirin001,Nelson Liriano,10,lee-m001,Manuel Lee,4,wilsw001,Willie Wilson,8,seitk001,Kevin Seitzer,5,bretg001,George Brett,3,tartd001,Danny Tartabull,9,tablp001,Pat Tabler,10,jackb001,Bo Jackson,7,boonb001,Bob Boone,2,whitf001,Frank White,4,stilk001,Kurt Stillwell,6,1,TOR,KCA,7,0.278,0.265,0.306,0.294,6,5,15,13,0.417,0.3

In [3]:
games.dtypes

date                  int64
num_of_games          int64
day                  object
visitor_team         object
v_league            float64
v_game_num            int64
home_team            object
h_league              int64
h_game_num            int64
v_runs                int64
h_runs                int64
total_outs            int64
day_night             int64
park_id              object
v_abs                 int64
v_hits                int64
v_2b                  int64
v_3b                  int64
v_hr                  int64
v_rbis                int64
v_sacbunt             int64
v_sacfly              int64
v_hbp                 int64
v_bb                  int64
v_ibb                 int64
v_k                   int64
v_sb                  int64
v_cs                  int64
v_gidp                int64
v_ci                  int64
v_lob                 int64
v_p_used              int64
v_ind_er              int64
v_team_er             int64
v_wildp               int64
v_balk              

In [4]:
X = games[['v_runs', 'h_runs', 'total_outs', 'v_abs','v_hits','v_2b','v_3b','v_hr','v_rbis','v_sacbunt','v_sacfly','v_hbp','v_bb','v_ibb','v_k','v_sb',
           'v_cs','v_gidp','v_ci','v_lob','v_p_used','v_ind_er','v_team_er','v_wildp','v_balk','v_putouts','v_assts','v_errors','v_passed_ball','v_dp',
           'v_trip','h_abs','h_hits','h_2b','h_3b','h_hr','h_rbis','h_sacbunt','h_sacfly','h_hbp','h_bb','h_ibb','h_k','h_sb','h_cs','h_gidp','h_ci',
           'h_lob','h_p_used','h_ind_er','h_team_er','h_wildp','h_balk','h_putouts','h_assts','h_errors','h_passed_ball','h_dp','h_trip','run_diff', 
           'W', 'L', 'total']]
y = games['run_diff']

In [5]:
categorical_columns = ['W', 'L']

# Create a ColumnTransformer with one-hot encoding
ct = ColumnTransformer(
    transformers=[
        ('oh', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), categorical_columns)
    ],
    remainder='passthrough'
)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
X_ct_train = ct.fit_transform(X_train)
X_ct_test = ct.transform(X_test)

In [9]:
model = RandomForestRegressor()
model.fit(X_ct_train, y_train)

RandomForestRegressor()

In [10]:
model.score(X_ct_test, y_test)

0.9999977921231894

In [11]:
preds = model.predict(X_ct_test)
preds

array([1., 2., 7., ..., 7., 5., 2.])

In [12]:
cv_scores = cross_val_score(model, X_ct_train, y_train, cv=5)

In [13]:
cv_scores

array([0.99961481, 0.99999977, 0.99999704, 0.99999852, 0.99999967])

In [14]:
np.mean(cv_scores)

0.999921961967509